# Scratchpad for personal exploration

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.dataset import ConnTextULDataset
from src.model import Model
import torch as torch

[nltk_data] Error loading cmudict: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


In [3]:
import pandas as pd
df = pd.read_csv("data/data_test50.csv")
df.shape

(50, 1)

In [4]:
for size in [20, 50, 100, 250, 500, 1000]:
    new_df = df['word_raw'][torch.randint(low=0, high=50000, size=(size,)).tolist()]
    new_df.to_csv(f"data/data_test{size}.csv", index=False)

KeyError: "None of [Int64Index([48508, 10326, 14046, 25414, 24923, 39616, 20705, 14472, 30073,\n            27181, 42380, 31641, 45987, 13295,  9703, 48658, 38992, 29223,\n            31962,  6268],\n           dtype='int64')] are in the [index]"

In [18]:
new_df.to_csv("data/data_test.csv", index=False)

In [93]:
"""
ConnTextULDataset should have option to read less rows
should test: nb_rows, test, which_dataset
Test == True: 
  - Use which_data (nb_rows should be disabled)
Test == False
  - Randomly sample nb_rows. If nb_rows==None, read the entire dataset

Suggestion: add a shuffle=Bool to CnnTextULDataset constructor when test=False

Comment: vocabularies of orthography and phonology are independent of the input data size. Somehow,
I don't think that should be the case. It could mess up testing. 
"""

ds = ConnTextULDataset(nb_rows=None, test=False, which_dataset=5)
num_layers = 2
num_layers_dict={
    'orth_enc': num_layers,
    'orth_dec': num_layers,
    'phon_enc': num_layers,
    'phon_dec': num_layers,
    'mixing_enc': num_layers
}
d_model = 64
m = Model(orth_vocab_size=len(ds.character_tokenizer),
          phon_vocab_size=len(ds.phonology_tokenizer),
          d_model=d_model,
          d_embedding=1,  # for mixing
          max_orth_seq_len=2,
          max_phon_seq_len=2,
          nhead=4,
          num_layers_dict=num_layers_dict
          )
m.eval();  # returns self (i.e., m)
orth_vocab_size = len(ds.character_tokenizer)
phon_vocab_size = len(ds.phonology_tokenizer)
print("orth_vocab_size: ", orth_vocab_size)
print("phon_vocab_size: ", phon_vocab_size)
print("d_model: ", d_model)
print("ds.dataset.shape: ", ds.dataset.shape)

ConnText..., nb_rows:  None
ConnText..., which_dataset:  5
CURRENT FOLDER:  /Users/erlebach/src/2023/ConnTextUL
len(self.dataset):  (90544, 7)
orth_vocab_size:  49
phon_vocab_size:  34
d_model:  64
ds.dataset.shape:  (90544, 7)


In [94]:
print(list(ds.__dict__.keys()))
print(ds.dataset.shape)

['dataset', 'phonology_tokenizer', 'character_tokenizer', 'max_orth_seq_len', 'max_phon_seq_len', 'max_seq_len', 'words']
(90544, 7)


In [95]:
# self.vocab = ['[BOS]', '[EOS]', '[CLS]', '[UNK]', '[PAD]']
# Ortho ids
#      BOS: 0
#      EOS: 1
#      CLS: 2
#      UNK: 3
#      PAD: 4
# Phono Ids
#    BOS: 31  (0 0 0 0 ... 1 0 0)   (a 1 in 31st position)
#    EOS: 32  (0 0 0 0 ... 0 1 0)
#    PAD: 33  (0 0 0 0 ... 0 0 1)

idx = 10040  # What is idx?phon['enc_input_ids'].shape, 
idx = 10031
num_words = 5   # batch_size
word = ds.words[idx:idx+num_words]
print("words = ", word)
batch = ds[idx:idx+num_words]
#print("batch = ", batch)
orth, phon = batch['orthography'].to('cpu'), batch['phonology'].to('cpu')
# orth and phon are dicitonaries
# ids represent either a character or a phoneme
# Each word: 0,x,x,x,1,4  (padding is optional. 0 and 1 are mandatory)
print("orth['enc_input_ids'] = ", orth['enc_input_ids'].shape, orth['enc_input_ids'])  # batch_size, max_orth_size
print("orth['enc_pad_mask'] = ", orth['enc_pad_mask'].shape, orth['enc_pad_mask'])
# Each word: 0,x,x,x,4  (padding is optional. 0 is  mandatory) No EoS on decoder input. Why? 
# Only input into the decoder a character if the next character will be predicted. 
# Since one does not predict the character folling EoS, it does not have to be in the decoder input
print("orth['dec_input_ids'] = ", orth['dec_input_ids'].shape, orth['dec_input_ids'])
print("orth['dec_pad_mask'] = ", orth['dec_pad_mask'].shape, orth['dec_pad_mask'])
#print("phon['enc_input_ids'] = ", phon['enc_input_ids'].shape, phon['enc_input_ids'])
# phon: no "bOS and EOS" tokens. Why not? 
print("==> phon['enc_input_ids'] = ", len(phon['enc_input_ids']))
# Each phono is a list of integers: e.g. [3,5] corresponds to [0 0 0 1 0 1 0 0 0 0 ... 0 0 ]
#                                   e.g. [0,3,5] corresponds to [1 0 0 1 0 1 0 0 0 0 ... 0 0 ]
for  i in phon['enc_input_ids']:
    print("    ", i)

print("phon['enc_pad_mask'] = ", phon['enc_pad_mask'].shape, phon['enc_pad_mask'])
#print("phon['dec_input_ids'] = ", phon['dec_input_ids'].shape, phon['dec_input_ids'])
print("==> phon['dec_input_ids'] = ", len(phon['dec_input_ids']))
for i in phon['dec_input_ids']:
    print("     ", i)
print("==> phon['dec_input_ids'] = ", len(phon['dec_input_ids']))
print("phon['dec_pad_mask'] = ", phon['dec_pad_mask'].shape, phon['dec_pad_mask'])
print("phon['targets'] = ", phon['targets'].shape, phon['targets'])

words =  ['as', 'she', 'and', 'you', 'was']
orth['enc_input_ids'] =  torch.Size([5, 5]) tensor([[ 0, 11, 29,  1,  4],
        [ 0, 29, 18, 15,  1],
        [ 0, 11, 24, 14,  1],
        [ 0, 35, 25, 31,  1],
        [ 0, 33, 11, 29,  1]])
orth['enc_pad_mask'] =  torch.Size([5, 5]) tensor([[False, False, False, False,  True],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False]])
orth['dec_input_ids'] =  torch.Size([5, 4]) tensor([[ 0, 11, 29,  4],
        [ 0, 29, 18, 15],
        [ 0, 11, 24, 14],
        [ 0, 35, 25, 31],
        [ 0, 33, 11, 29]])
orth['dec_pad_mask'] =  torch.Size([5, 4]) tensor([[False, False, False,  True],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]])
==> phon['enc_input_ids'] =  5
     [tensor([31]), tensor([14, 15, 17, 22, 29]), te

In [96]:
z = m(orth_enc_input=orth['enc_input_ids'],
      orth_enc_pad_mask=orth['enc_pad_mask'],
      orth_dec_input=orth['dec_input_ids'],
      orth_dec_pad_mask=orth['dec_pad_mask'],
      phon_enc_input=phon['enc_input_ids'],
      phon_enc_pad_mask=phon['enc_pad_mask'],
      phon_dec_input=phon['dec_input_ids'],
      phon_dec_pad_mask=phon['dec_pad_mask']
      )

before mixed_encoding
enter embed_tokens
tokens shape:  torch.Size([5, 5])
tokens:  tensor([[ 0, 11, 29,  1,  4],
        [ 0, 29, 18, 15,  1],
        [ 0, 11, 24, 14,  1],
        [ 0, 35, 25, 31,  1],
        [ 0, 33, 11, 29,  1]])
=shape self.orthography_embedding(tokens):  torch.Size([5, 5, 64])
=shape self.orth_position_embedding.weight[None, :tokens.shape[1]]:  torch.Size([1, 2, 64])


RuntimeError: The size of tensor a (5) must match the size of tensor b (2) at non-singleton dimension 1

In [92]:
print("z['orth'].shape", z['orth'].shape)
print("z['orth'] = ", z['orth'])
print("orth['enc_input_ids'].shape = ", orth['enc_input_ids'].shape)
labels = orth['enc_input_ids'][:,1:]
print("orth['enc_input_ids'][:,1:] = ", labels)
preds = torch.argmax(z['orth'], dim=1)
print("torch.argmax(z['orth'], dim=1) = ", preds)
accuracy = (preds == labels).sum() / len(labels)
print("accuracy = ", accuracy)

NameError: name 'z' is not defined

In [38]:
preds == labels

tensor([[False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [ True, False, False, False, False],
        [False, False, False, False, False]])

In [23]:
print("z['phon'].shape", z['phon'].shape)
print("z['phon'] = ", z['phon'])
torch.argmax(z['phon'], dim=1)

z['phon'].shape torch.Size([1, 2, 4, 33])
z['phon'] =  tensor([[[[ 4.4194e-01,  1.2137e-02, -1.1275e+00, -1.3524e-01,  6.3349e-02,
           -2.1050e-01, -9.0739e-02,  3.0333e-01,  5.6287e-01, -1.5716e-03,
           -4.5574e-01, -8.8808e-01, -7.1200e-01, -3.6442e-01,  7.7940e-02,
           -1.1434e-01,  3.8295e-01, -5.4414e-01,  4.1145e-01,  2.9466e-01,
            3.3463e-01, -2.9968e-02, -7.9627e-01,  1.0003e+00, -1.3511e-01,
            1.0474e+00, -7.4798e-02,  2.0223e-01, -2.5267e-01,  4.4629e-01,
           -2.7202e-02, -4.7697e-01, -4.0632e-01],
          [ 7.3698e-01,  8.1904e-02,  6.9783e-01, -1.5817e-01,  8.8913e-01,
           -3.7205e-01, -1.2592e-01,  1.2707e-01,  1.7307e-01,  2.5315e-01,
           -1.6076e-01,  3.8544e-01, -5.7465e-01, -7.1618e-01, -8.9231e-01,
           -5.9913e-01, -3.8458e-01,  9.2893e-02,  1.3562e-01,  4.3413e-01,
            1.0148e-01,  1.6369e-01, -1.5533e-01,  3.9790e-01, -1.9901e-02,
            3.8274e-01,  9.4589e-02,  4.4247e-01, -7.1658e

tensor([[[1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
          0, 0, 0, 1, 0, 1, 0, 1, 1, 0],
         [0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
          0, 0, 0, 1, 0, 1, 0, 1, 1, 1],
         [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
          0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
         [0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0,
          1, 0, 0, 0, 0, 1, 1, 0, 1, 0]]])

In [10]:
orth

tensor([[0]])

In [ ]:
logits = m(orthography['enc_input_ids'], orthography['enc_pad_mask'],
                orthography['dec_input_ids'], orthography['dec_pad_mask'],
                phonology['enc_input_ids'], phonology['enc_pad_mask'],
                phonology['dec_input_ids'], phonology['dec_pad_mask'])
        
orth_loss = pt.nn.CrossEntropyLoss(ignore_index=4)(logits['orth'], orthography['enc_input_ids'][:,1:]) 
phon_loss = pt.nn.CrossEntropyLoss(ignore_index=2)(logits['phon'], phonology['targets'])
loss = orth_loss + phon_loss

In [57]:
z_prime = m.generate(orth_enc_input=orth, 
                     orth_enc_pad_mask=torch.zeros_like(orth).bool(), 
                     phon_enc_input=phon, 
                     phon_enc_pad_mask=torch.zeros_like(phon).bool(), 
                     deterministic=True)['orth'][:, 1:]
z_prime


phon_output.shape =  torch.Size([1, 1, 64])
phonology_token_logits.shape =  torch.Size([1, 1, 66])
phonology_token_logits.shape =  torch.Size([1, 2, 1, 33])
phon_output.shape =  torch.Size([1, 2, 64])
phonology_token_logits.shape =  torch.Size([1, 2, 66])
phonology_token_logits.shape =  torch.Size([1, 2, 2, 33])
phon_output.shape =  torch.Size([1, 3, 64])
phonology_token_logits.shape =  torch.Size([1, 3, 66])
phonology_token_logits.shape =  torch.Size([1, 2, 3, 33])
phon_output.shape =  torch.Size([1, 4, 64])
phonology_token_logits.shape =  torch.Size([1, 4, 66])
phonology_token_logits.shape =  torch.Size([1, 2, 4, 33])
phon_output.shape =  torch.Size([1, 5, 64])
phonology_token_logits.shape =  torch.Size([1, 5, 66])
phonology_token_logits.shape =  torch.Size([1, 2, 5, 33])
phon_output.shape =  torch.Size([1, 6, 64])
phonology_token_logits.shape =  torch.Size([1, 6, 66])
phonology_token_logits.shape =  torch.Size([1, 2, 6, 33])
phon_output.shape =  torch.Size([1, 7, 64])
phonology_toke

tensor([[16, 40,  4,  0, 16, 15,  5, 45,  7, 31, 19, 16, 42, 40,  8, 15,  0, 16,
         42]])

In [28]:

#assert z.item() == z_prime.item()

################# Check Phonology ########################
z = m(orth, 
      torch.zeros_like(orth).bool(), 
      orth, 
      torch.zeros_like(orth).bool(),
      [[phon][0]], 
      torch.zeros_like(phon).bool(), 
      [[phon][0]], 
      torch.zeros_like(phon).bool()
)['phon']; z


tensor([[[[ 0.6767, -0.1219, -0.5411,  0.1170, -0.3387, -0.1288, -0.5503,
            0.6902,  0.1972, -0.2650,  0.5467,  0.9316,  0.4024, -0.4422,
           -0.1477,  0.7241, -0.9001, -0.2532,  0.5851,  1.1965, -0.7901,
            0.2979, -0.0242,  0.4706,  0.8686,  0.1165,  0.4917,  0.5010,
           -0.1539, -0.0028,  0.1609, -0.2441,  0.3435]],

         [[ 0.5110,  0.4595, -0.6616,  0.0228, -0.6862,  0.1313,  0.8328,
            0.2021,  0.2570,  0.8814,  0.1321, -0.1988, -0.1504, -0.1617,
            0.7846, -0.2460,  0.9513, -0.4330, -0.6739,  0.0562,  0.5938,
            0.0119, -0.3092, -0.0674,  0.9380,  0.5049,  0.0826, -0.9075,
            0.3517,  0.6457,  0.2784, -0.3076,  0.8851]]]],
       grad_fn=<TransposeBackward0>)

In [39]:

z_prime = m.generate(orth, 
                     torch.zeros_like(orth).bool(), 
                     [[phon][0]], 
                     torch.zeros_like(phon).bool(), 
                     deterministic=True)['phon']
z_prime

#assert (torch.where(z[0].argmax(dim=0) == 1)[1] == z_prime[0][1]).all()
print(len(z_prime[0]))

20


In [42]:
B,C,E = 2,3,4
a = torch.rand((B,C,E))

In [44]:
print(a.shape)
print(a.transpose(1,2).shape)

torch.Size([2, 3, 4])
torch.Size([2, 4, 3])


In [62]:
p=torch.rand((1,2,33)); p

tensor([[[0.4301, 0.9611, 0.7723, 0.9204, 0.1864, 0.3543, 0.0298, 0.9956,
          0.0215, 0.8572, 0.2218, 0.4789, 0.0823, 0.7532, 0.1532, 0.0425,
          0.4336, 0.8475, 0.8205, 0.8800, 0.2938, 0.7716, 0.4422, 0.3135,
          0.1729, 0.0260, 0.9826, 0.9753, 0.2033, 0.6105, 0.4312, 0.0927,
          0.9542],
         [0.7559, 0.8657, 0.4542, 0.3938, 0.1533, 0.3608, 0.4802, 0.3396,
          0.7139, 0.2800, 0.1930, 0.4848, 0.7758, 0.4037, 0.2637, 0.3022,
          0.6368, 0.4707, 0.4163, 0.9229, 0.5459, 0.4463, 0.9723, 0.4306,
          0.7044, 0.4951, 0.3131, 0.5019, 0.4948, 0.4797, 0.4198, 0.3487,
          0.9346]]])

In [78]:
torch.bernoulli(p[:,1,:])

tensor([[1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
         1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1.]])

In [70]:
torch.bernoulli(p)[0]

tensor([[1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
         1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.],
        [1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
         0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1.]])

In [80]:
p[:,1]

tensor([[0.7559, 0.8657, 0.4542, 0.3938, 0.1533, 0.3608, 0.4802, 0.3396, 0.7139,
         0.2800, 0.1930, 0.4848, 0.7758, 0.4037, 0.2637, 0.3022, 0.6368, 0.4707,
         0.4163, 0.9229, 0.5459, 0.4463, 0.9723, 0.4306, 0.7044, 0.4951, 0.3131,
         0.5019, 0.4948, 0.4797, 0.4198, 0.3487, 0.9346]])

In [84]:
torch.where(p[:,1] > 0.5)[1]

tensor([ 0,  1,  8, 12, 16, 19, 20, 22, 24, 27, 32])

In [173]:
chkpt = torch.load('models/model0_checkpoint99.pth', map_location=torch.device('cpu'))
m = Model(len(ds.character_tokenizer), len(ds.phonology_tokenizer), d_model=chkpt['d_model'], nhead=chkpt['nhead'])
m.load_state_dict(chkpt['model'])
m.eval();

In [178]:
idx = 10040
word = ds.words[idx:idx+1]
print("word = ", word)
batch = ds[idx:idx+1]
#print("batch = ", batch)
orth, phon = batch['orthography'].to('cpu'), batch['phonology'].to('cpu')
orthography = orth['enc_input_ids']
orthography_mask = orth['enc_pad_mask']
phonology = phon['enc_input_ids']
phonology_mask = phon['enc_pad_mask']

word =  ['sam']


In [179]:
generation = m.generate(orthography, orthography_mask, phonology, phonology_mask, deterministic=False)

In [180]:
ds.character_tokenizer.decode(generation['orth'][:,:].tolist())

['[BOS]ci”γ[EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS]γ[EOS]γ']

In [181]:
generation['phon'][0]

[tensor([31]),
 tensor([ 9, 14]),
 tensor([14, 15, 17, 26, 28, 29]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([14, 18]),
 tensor([32]),
 tensor([32])]

In [ ]:
ds.phonology_tokenizer.decode(generation['phon'][0])

In [148]:
print(ds.character_tokenizer.vocab)

['[BOS]', '[EOS]', '[CLS]', '[UNK]', '[PAD]', 'i', '’', '\x80', 'o', 'f', 'โ', 'v', '“', '.', ',', 'r', "'", '\x9d', '\x9c', 'g', 'k', 'c', 'n', 'm', 'z', '-', 'y', 'e', 'h', 'j', 'p', 'x', 's', '"', 'q', '\x94', 'ç', '¥', 'w', '”', 'l', 'a', '!', 'u', 'b', 'd', 't', '—', 'γ']


In [158]:
ds

['__add__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_protocol',
 'character_tokenizer',
 'dataset',
 'phonology_tokenizer',
 'words']